# Cost of Living (CoL) Web Scraper

## Required Packages

In [1]:
library(rvest)
library(tidyverse)
library(stringr)

Loading required package: xml2
Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


## Reading the webpage
Reading the webpage and aiming for the data_wide_table selector, this is the 3rd returned table from html_nodes below
![Numbeo Page](./images/numbeo_swazi.png)

In [2]:
CoL_page = read_html("https://www.numbeo.com/cost-of-living/country_result.jsp?country=Swaziland") #get html from page
CoL_elements = html_nodes(x = CoL_page, css = "table") #get all the tables on the pages
CoL_text = html_text(CoL_elements[3], trim = TRUE) #3rd table returned\n",
CoL_table = as_tibble(html_table(CoL_elements[3])[[1]])#removing table from list
head(CoL_table)

Restaurants,[ Edit ],Range
"Meal, Inexpensive Restaurant",70.00 R,50.00-70.00
"Meal for 2 People, Mid-range Restaurant, Three-course",300.00 R,300.00-400.00
McMeal at McDonalds (or Equivalent Combo Meal),50.00 R,50.00-59.00
Domestic Beer (0.5 liter draught),13.00 R,11.00-15.00
Imported Beer (0.33 liter bottle),17.50 R,12.00-20.00
Cappuccino (regular),17.50 R,15.00-20.00


In [3]:
colnames(CoL_table) = c("item", 'avgPrice', 'range') #rename the columns
CoL_table = CoL_table %>% filter(avgPrice != "[ Edit ]") #remove categories that corrupt table
CoL_table %>% head(5)

item,avgPrice,range
"Meal, Inexpensive Restaurant",70.00 R,50.00-70.00
"Meal for 2 People, Mid-range Restaurant, Three-course",300.00 R,300.00-400.00
McMeal at McDonalds (or Equivalent Combo Meal),50.00 R,50.00-59.00
Domestic Beer (0.5 liter draught),13.00 R,11.00-15.00
Imported Beer (0.33 liter bottle),17.50 R,12.00-20.00


## "Borrowing" Countries
Using the Incpect Element function of google chrome, "borrowing" the countries from the numbeo home page
![Borrowing Coutnries](./images/strealCountries.png)

In [4]:
countries = c("Afghanistan","Aland Islands","Albania","Algeria"
              ,"Andorra","Angola","Antigua And Barbuda","Argentina"
              ,"Armenia","Aruba","Australia","Austria","Azerbaijan"
              ,"Bahamas","Bahrain","Bangladesh","Barbados","Belarus"
              ,"Belgium","Belize","Bermuda","Bhutan","Bolivia"
              ,"Bosnia And Herzegovina","Botswana","Brazil"
              ,"British Virgin Islands","Brunei","Bulgaria","Burkina Faso"
              ,"Burundi","Cambodia","Cameroon","Canada","Cape Verde"
              ,"Cayman Islands","Chad","Chile","China","Colombia","Congo"
              ,"Costa Rica","Croatia","Cuba","Curacao","Cyprus","Czech Republic"
              ,"Denmark","Djibouti","Dominica","Dominican Republic","Ecuador"
              ,"Egypt","El Salvador","Estonia","Ethiopia","Faroe Islands"
              ,"Fiji","Finland","France","French Polynesia","Gabon","Gambia"
              ,"Georgia","Germany","Ghana","Gibraltar","Greece","Greenland"
              ,"Grenada","Guam","Guatemala","Guernsey","Guyana","Honduras"
              ,"Hong Kong","Hungary","Iceland","India","Indonesia","Iran"
              ,"Iraq","Ireland","Isle Of Man","Israel","Italy","Ivory Coast"
              ,"Jamaica","Japan","Jersey","Jordan","Kazakhstan","Kenya"
              ,"Kosovo (Disputed Territory)","Kuwait","Kyrgyzstan","Laos"
              ,"Latvia","Lebanon","Lesotho","Liberia","Libya","Lithuania"
              ,"Luxembourg","Macao","Macedonia","Madagascar","Malawi"
              ,"Malaysia","Maldives","Mali","Malta","Mauritania","Mauritius"
              ,"Mexico","Micronesia","Moldova","Monaco","Mongolia","Montenegro"
              ,"Morocco","Mozambique","Myanmar","Namibia","Nepal","Netherlands"
              ,"New Caledonia","New Zealand","Nicaragua","Nigeria","Northern Mariana Islands"
              ,"Norway","Oman","Pakistan","Palestinian Territory","Panama","Papua New Guinea"
              ,"Paraguay","Peru","Philippines","Poland","Portugal","Puerto Rico","Qatar"
              ,"Reunion","Romania","Russia","Rwanda","Saint Kitts And Nevis","Saint Lucia"
              ,"Saint Vincent And The Grenadines","Samoa","Saudi Arabia","Senegal","Serbia"
              ,"Seychelles","Singapore","Sint Maarten","Slovakia","Slovenia","Somalia","South Africa"
              ,"South Korea","South Sudan","Spain","Sri Lanka","Sudan","Suriname","Swaziland"
              ,"Sweden","Switzerland","Syria","Taiwan","Tajikistan","Tanzania","Thailand","Timor-Leste"
              ,"Togo","Tonga","Trinidad And Tobago","Tunisia","Turkey","Turkmenistan","Turks And Caicos Islands"
              ,"Uganda","Ukraine","United Arab Emirates","United Kingdom","United States"
              ,"Uruguay","Us Virgin Islands","Uzbekistan","Vanuatu","Venezuela","Vietnam"
              ,"Yemen","Zambia","Zimbabwe")

Noticing that the URL for countries with spaces are addressed with +'s
![Spaces URL address space](./images/spaceURL.png)

## The Crawl Function
This is the function that takes a country and create the URL needed to get the table for that country

In [5]:
getCoL = function(country){

    now = Sys.time()

    writeLines(str_c(now, "Scraping ", country, sep="\t"))
    URL = str_c("https://www.numbeo.com/cost-of-living/country_result.jsp?country=", country, sep="")
    #get html from page
    CoL_page = read_html(URL) 
    #get all the tables on the pages
    CoL_elements = html_nodes(x = CoL_page, css = "table") 
    #3rd table returned
    CoL_text = html_text(CoL_elements[3], trim = TRUE) 
    #removing table from list
    CoL_table = as_tibble(html_table(CoL_elements[3])[[1]])
    #rename the columns
    colnames(CoL_table) = c("item", 'avgPrice', 'range') 
    #remove categories that corrupt table
    CoL_table = CoL_table %>% filter(avgPrice != "[ Edit ]") 
    #split the numbers out of the avgPrice column to identify currency used,
    #     currency_pass_1 = CoL_table %>% select(avgPrice) %>% unlist() %>% str_split("[0-9]") ,
    #     currency = str_split(currency_pass_1[[1]], " ") %>% tail(1) %>% unlist()
    writeLines(paste0("Time:\t\t ", Sys.time() -now))
    return(CoL_table)
}

In [6]:
now = Sys.time()
countries_plus = lapply(countries[1:20], function(x) gsub(" ", "+", x, fixed=TRUE))
country_tibbles = lapply(countries_plus, function(x) getCoL(x))
Sys.time()-now

2017-08-21 13:23:33	Scraping 	Afghanistan
Time:		 1.17964100837708
2017-08-21 13:23:34	Scraping 	Aland+Islands
Time:		 1.18362998962402
2017-08-21 13:23:35	Scraping 	Albania
Time:		 1.17282485961914
2017-08-21 13:23:36	Scraping 	Algeria
Time:		 1.19793105125427
2017-08-21 13:23:38	Scraping 	Andorra
Time:		 1.17823481559753
2017-08-21 13:23:39	Scraping 	Angola
Time:		 1.20951199531555
2017-08-21 13:23:40	Scraping 	Antigua+And+Barbuda
Time:		 1.18173599243164
2017-08-21 13:23:41	Scraping 	Argentina
Time:		 1.18727993965149
2017-08-21 13:23:42	Scraping 	Armenia
Time:		 1.17934107780457
2017-08-21 13:23:44	Scraping 	Aruba
Time:		 1.18879008293152
2017-08-21 13:23:45	Scraping 	Australia
Time:		 1.25242280960083
2017-08-21 13:23:46	Scraping 	Austria
Time:		 1.17434692382812
2017-08-21 13:23:47	Scraping 	Azerbaijan
Time:		 1.2283570766449
2017-08-21 13:23:48	Scraping 	Bahamas
Time:		 1.1672899723053
2017-08-21 13:23:50	Scraping 	Bahrain
Time:		 1.18383884429932
2017-08-21 13:23:51	Scraping 	B

Time difference of 23.93127 secs

In [12]:
#finding a few countries with the same currency
for(i in 1:20){print(paste0(i," - ",country_tibbles[[i]]$avgPrice[1]))}

[1] "1 - 3.67 $"
[1] "2 - 10.40 €"
[1] "3 - 661.83 Lek"
[1] "4 - 300.00 DZD"
[1] "5 - 10.00 €"
[1] "6 - 19.29 $"
[1] "7 - 25.00 EC$"
[1] "8 - 160.00 ARS"
[1] "9 - 3,000.00 AMD"
[1] "10 - 11.00 $"
[1] "11 - 18.00 A$"
[1] "12 - 10.00 €"
[1] "13 - 8.00 man"
[1] "14 - 16.50 $"
[1] "15 - 2.00 BHD"
[1] "16 - 180.00 ৳"
[1] "17 - 25.00 Bds$"
[1] "18 - 6.83 $"
[1] "19 - 15.00 €"
[1] "20 - 4.00 $"


In [15]:
euro_countries = c(2,5,12,19)
print(countries[euro_countries])

[1] "Aland Islands" "Andorra"       "Austria"       "Belgium"      


In [25]:
print("Aland Islands")
country_tibbles[2][[1]][1:10,]

print("Andorra")
country_tibbles[2][[1]][1:10,]

print("Austria")
country_tibbles[2][[1]][1:10,]

print("Belgium")
country_tibbles[2][[1]][1:10,]

[1] "Aland Islands"


item,avgPrice,range
"Meal, Inexpensive Restaurant",10.40 €,10.00-14.00
"Meal for 2 People, Mid-range Restaurant, Three-course",70.00 €,52.00-80.00
McMeal at McDonalds (or Equivalent Combo Meal),7.95 €,7.90-8.00
Domestic Beer (0.5 liter draught),6.00 €,5.00-6.00
Imported Beer (0.33 liter bottle),5.00 €,3.00-7.00
Cappuccino (regular),3.00 €,2.00-4.00
Coke/Pepsi (0.33 liter bottle),1.40 €,1.00-2.50
Water (0.33 liter bottle),1.50 €,0.50-2.50
"Milk (regular), (1 liter)",1.15 €,1.00-1.30
Loaf of Fresh White Bread (500g),2.70 €,1.40-4.00


[1] "Andorra"


item,avgPrice,range
"Meal, Inexpensive Restaurant",10.40 €,10.00-14.00
"Meal for 2 People, Mid-range Restaurant, Three-course",70.00 €,52.00-80.00
McMeal at McDonalds (or Equivalent Combo Meal),7.95 €,7.90-8.00
Domestic Beer (0.5 liter draught),6.00 €,5.00-6.00
Imported Beer (0.33 liter bottle),5.00 €,3.00-7.00
Cappuccino (regular),3.00 €,2.00-4.00
Coke/Pepsi (0.33 liter bottle),1.40 €,1.00-2.50
Water (0.33 liter bottle),1.50 €,0.50-2.50
"Milk (regular), (1 liter)",1.15 €,1.00-1.30
Loaf of Fresh White Bread (500g),2.70 €,1.40-4.00


[1] "Austria"


item,avgPrice,range
"Meal, Inexpensive Restaurant",10.40 €,10.00-14.00
"Meal for 2 People, Mid-range Restaurant, Three-course",70.00 €,52.00-80.00
McMeal at McDonalds (or Equivalent Combo Meal),7.95 €,7.90-8.00
Domestic Beer (0.5 liter draught),6.00 €,5.00-6.00
Imported Beer (0.33 liter bottle),5.00 €,3.00-7.00
Cappuccino (regular),3.00 €,2.00-4.00
Coke/Pepsi (0.33 liter bottle),1.40 €,1.00-2.50
Water (0.33 liter bottle),1.50 €,0.50-2.50
"Milk (regular), (1 liter)",1.15 €,1.00-1.30
Loaf of Fresh White Bread (500g),2.70 €,1.40-4.00


[1] "Belgium"


item,avgPrice,range
"Meal, Inexpensive Restaurant",10.40 €,10.00-14.00
"Meal for 2 People, Mid-range Restaurant, Three-course",70.00 €,52.00-80.00
McMeal at McDonalds (or Equivalent Combo Meal),7.95 €,7.90-8.00
Domestic Beer (0.5 liter draught),6.00 €,5.00-6.00
Imported Beer (0.33 liter bottle),5.00 €,3.00-7.00
Cappuccino (regular),3.00 €,2.00-4.00
Coke/Pepsi (0.33 liter bottle),1.40 €,1.00-2.50
Water (0.33 liter bottle),1.50 €,0.50-2.50
"Milk (regular), (1 liter)",1.15 €,1.00-1.30
Loaf of Fresh White Bread (500g),2.70 €,1.40-4.00


To my suprise all of the prices are the same... 🤦‍. This can be improved by scraping currencies and comparing the countries to a universal metric